[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/ik-nlp-tutorials/blob/main/notebooks/W5E_Inseq_Analysis.ipynb)

In [2]:
# Run in Colab to install local packages
!pip install inseq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.6/186.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

# Exercise 1: Analyzing language generation models with Inseq 🐛

*Adapted in part from the [Inseq documentation](https://inseq.readthedocs.io/)*

Inseq is a toolkit based on the 🤗 Transformers and [Captum](https://captum.ai/docs/introduction) libraries for interpreting language generation models using feature attribution methods. Inseq allows you to analyze the behavior of a language generation model by computing the importance of each input token for each token in the generated output. The importance can be obtained using approaches based on attention, gradients and more, which we will see in more detail in the final lecture.

Inseq is a relatively new library, and it is still under active development (contributions welcome! 🙂). You can refer to the [Inseq paper](https://arxiv.org/abs/2302.13942) for an overview of the tool and some examples, or [this paper](https://www.semanticscholar.org/paper/Are-Character-level-Translations-Worth-the-Wait-An-Edman-Toral/ed7b51e4a5c4835218f6697b280afb2849211939) for a recent work from our GroNLP group on using Inseq to analyze character-level translation models.

In the following sections two simple use-cases of Inseq are presented.

## Attributing (Un)constrained Machine Translation

In this section we will use Inseq to compute the importance of each input token for each token in the generated output. We will use the [Helsinki-NLP/opus-mt-en-nl](https://huggingface.co/Helsinki-NLP/opus-mt-en-nl) model, which is a pretrained machine translation model from English to Dutch.

In [2]:
import inseq
import warnings

warnings.filterwarnings("ignore")

model = inseq.load_model("Helsinki-NLP/opus-mt-en-nl", "input_x_gradient")
out = model.attribute(
    "Don't get hot-headed mate, it's easy peasy to study models with Inseq!",
    attribute_target=True,
    step_scores=["probability"]
)
out.show()

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/316M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/316M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Attributing with input_x_gradient...: 100%|██████████| 20/20 [00:05<00:00,  3.49it/s]


,▁Maak,▁je,▁niet,▁zo,▁druk,",",▁het,▁is,▁makkelijk,▁om,▁modellen,▁te,▁bestuderen,▁met,▁In,se,q,!,</s>
▁Don,0.045,0.036,0.02,0.028,0.022,0.026,0.02,0.014,0.02,0.01,0.011,0.012,0.016,0.012,0.007,0.006,0.005,0.027,0.02
',0.019,0.017,0.013,0.017,0.011,0.016,0.012,0.007,0.016,0.007,0.007,0.008,0.011,0.006,0.005,0.004,0.003,0.025,0.017
t,0.031,0.025,0.028,0.021,0.017,0.02,0.018,0.01,0.009,0.005,0.006,0.007,0.008,0.006,0.008,0.004,0.002,0.014,0.012
▁get,0.118,0.068,0.056,0.05,0.049,0.027,0.03,0.02,0.014,0.01,0.013,0.012,0.015,0.012,0.01,0.005,0.004,0.018,0.021
▁hot,0.066,0.073,0.071,0.088,0.084,0.042,0.026,0.022,0.016,0.01,0.014,0.011,0.012,0.011,0.009,0.008,0.007,0.022,0.019
-,0.021,0.02,0.024,0.027,0.025,0.018,0.01,0.007,0.008,0.005,0.005,0.005,0.007,0.005,0.005,0.005,0.006,0.013,0.009
headed,0.143,0.146,0.179,0.136,0.178,0.133,0.068,0.051,0.029,0.023,0.034,0.027,0.047,0.032,0.021,0.013,0.009,0.041,0.042
▁mate,0.085,0.072,0.081,0.061,0.07,0.095,0.052,0.02,0.02,0.013,0.014,0.013,0.02,0.015,0.01,0.005,0.005,0.021,0.022
",",0.02,0.02,0.017,0.016,0.015,0.03,0.019,0.013,0.014,0.006,0.005,0.007,0.008,0.006,0.004,0.004,0.003,0.021,0.018
▁it,0.017,0.012,0.012,0.01,0.009,0.021,0.026,0.022,0.014,0.012,0.008,0.013,0.014,0.009,0.005,0.004,0.003,0.017,0.014


<details>
    <summary> Observations </summary>

- The model translates idiomatic expressions in a non-literal way. Attribution scores reflect that the model is attributing strong importance to pieces of the idiom when translating (e.g. `_headed`, `_peas`), while also accounting for the prefix when producing an idiomatic translation (e.g. `_Maak`).
- The model gets progressively more confident in its translation as it generates more tokens. The first generated token is very unlikely.
</details>

Let's try now to **constrain** the generation to a more literal translation of the input. Attributing a prespecified output can be intuitively thought as a way to ask a model to justify a possible prediction. Note that this should be done with care, since if the output is very unlikely the results will be very noisy.

In [3]:
import inseq

model = inseq.load_model("Helsinki-NLP/opus-mt-en-nl", "input_x_gradient")
out = model.attribute(
    "Don't get hot-headed mate, it's easy peasy to study models with Inseq!",
    #"Niet heethoofdig worden maatje, het is gemakkelijk peasy om modellen te bestuderen met Inseq!",
    "Niet heethoofdig worden man, het is een makkie om modellen te bestuderen met Inseq!",
    attribute_target=True,
    step_scores=["probability"]
)
out.show()

Attributing with input_x_gradient...: 100%|██████████| 24/24 [00:07<00:00,  3.11it/s]


,▁Niet,▁heet,hoofd,ig,▁worden,▁man,",",▁het,▁is,▁een,▁,mak,kie,▁om,▁modellen,▁te,▁bestuderen,▁met,▁In,se,q,!,</s>
▁Don,0.08,0.041,0.013,0.018,0.036,0.024,0.028,0.017,0.011,0.013,0.009,0.011,0.006,0.009,0.01,0.013,0.016,0.011,0.008,0.006,0.005,0.027,0.013
',0.031,0.025,0.012,0.015,0.024,0.016,0.024,0.015,0.006,0.008,0.005,0.006,0.004,0.007,0.007,0.008,0.011,0.006,0.004,0.004,0.003,0.026,0.01
t,0.045,0.021,0.013,0.013,0.024,0.02,0.018,0.011,0.007,0.007,0.006,0.005,0.003,0.005,0.006,0.007,0.008,0.005,0.007,0.004,0.002,0.014,0.009
▁get,0.095,0.041,0.018,0.034,0.071,0.042,0.025,0.019,0.013,0.011,0.009,0.011,0.006,0.008,0.011,0.01,0.015,0.01,0.01,0.005,0.004,0.02,0.021
▁hot,0.082,0.091,0.09,0.055,0.064,0.042,0.026,0.021,0.016,0.014,0.013,0.015,0.01,0.012,0.011,0.01,0.012,0.01,0.009,0.007,0.007,0.023,0.017
-,0.023,0.023,0.032,0.019,0.018,0.021,0.013,0.01,0.006,0.007,0.005,0.006,0.003,0.004,0.005,0.005,0.007,0.005,0.005,0.005,0.006,0.012,0.007
headed,0.126,0.186,0.263,0.143,0.092,0.145,0.071,0.06,0.039,0.032,0.033,0.04,0.02,0.026,0.032,0.027,0.046,0.03,0.022,0.013,0.009,0.041,0.039
▁mate,0.08,0.05,0.056,0.112,0.071,0.197,0.066,0.088,0.042,0.04,0.024,0.022,0.016,0.021,0.036,0.02,0.02,0.025,0.021,0.006,0.005,0.028,0.04
",",0.025,0.021,0.015,0.022,0.018,0.026,0.021,0.014,0.012,0.009,0.006,0.008,0.005,0.007,0.006,0.007,0.008,0.006,0.004,0.004,0.003,0.021,0.011
▁it,0.02,0.016,0.012,0.012,0.011,0.016,0.019,0.017,0.024,0.013,0.011,0.011,0.005,0.01,0.008,0.014,0.014,0.008,0.005,0.004,0.003,0.017,0.009


### Your turn to comment

Comment on the resulting scores from the constrained, less idiomatic example, putting them in relation to the unconstrained, more idiomatic one. Consider the following aspects, but feel free to explore other examples and add your own observations:
1. How are importance scores distributed on idiomatic and non-idiomatic tokens?
2. What is the difference in probability between the two examples?
3. Do you notice some patterns regarding the low-probability tokens in the second example?
4. When is the target prefix playing a more important role in the generation, according to the attribution scores?

**TODO**
In the idiomatic example the importance scores seem more evenly distributed and with more consistent probablities, whereas the literal translation has an uneven pattern where some the token probabilities vary greatly, which could suggest that the forced phrasing of the literal translation is less likely to occur naturally. The literal version also has many tokens that have almost no importance, suggesting that the model is not well suited to split them. Also, the target prefix does seem to play a strong role in guiding the idiomatic output, which is less noticeable in the literal translation

## Contrastive Attribution for Motivating Preferences

In the previous section we used importance scores produced by attributing next token’s probability, which can be seen as answering the question “Which elements of the input are the most relevant to produce the next generation step?”.

However, in many cases we might be more interested in understanding why our model generated its output **rather than another one that we consider to be more likely**. The paper [“Interpreting Language Models with Contrastive Explanations”](https://arxiv.org/abs/2202.10419) by Yin and Neubig (2022) proposes a contrastive attribution method that can be used to answer this question. The method is integrated in Inseq and can be used as follows:

In [ ]:
import inseq

model = inseq.load_model("google/flan-t5-base", "input_x_gradient")

out = model.attribute(
    "Does 3 + 3 equal 6?",
    # Fix the original target
    "yes",
    attributed_fn="contrast_prob_diff",
    # Also show the probability delta between the two options
    step_scores=["contrast_prob_diff", "probability"],
    contrast_targets="no",
)

# Normally attributions are normalized to sum up to 1
# Here we want to see how they contribute to the probability difference
out.show()

Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:00<00:00,  3.70it/s]


,▁no → ▁yes,</s>
▁Does,0.149,0.0
▁3,0.09,0.0
▁+,0.089,0.0
▁3,0.097,0.0
▁equal,0.191,0.0
▁6,0.154,0.0
?,0.11,0.0
</s>,0.12,0.0
contrast_prob_diff,0.05,-0.0
probability,0.523,1.0


We can see that the model is relying more heavily on formulation keywords (`Does`, `equal`) and less on the numbers to determine the answer. The gap between the positive and negative answer is also quite small (5%), suggesting that the model is not very confident in its answer. Changing the input to `Does 3 + 3 equal 7?` confirms that the actual expression is not playing a relevant role in the generation.

> ⚠️ **Important**: Since contrastive attribution compares the probabilities of a pair of `(original, contrastive)` tokens, in order for it to work, the compared sequences must have the same length. For example, if "yes" was tokenized as `_y`, `es` we couldn't have compared it with a single token `_no` using Inseq.

### Your turn to attribute

Using the generation model and a task of your choice, try to use contrastive attribution on at least three examples to highlight some interesting pattern of your choice. We encourage you to explore whathever you find most interesting, but here are some suggestions:

- Is negation relevant in producing the correct answer in open question answering models like the one we used in the previous example?

- When translating sentences like `The nurse went to the hospital` to a gendered language like Spanish, Italian or German, the model will have to select a gender for the subject. What is the model relying on to make this choice?

- Considering a fixed example like `Does 3 + 3 = 6?` but using models with increasingly more parameters (e.g. `flan-t5-small`, `flan-t5-base`, `flan-t5-large`), how does input importance and model confidence change?

After producing the visualizations, comment on the results and try to explain what you observe.

Inseq also supports attribution of quantized models (see [example](https://inseq.readthedocs.io/examples/locate_gpt2_knowledge.html)), in case you want to explore using larger models for your analysis. Refer to the [Inseq documentation](https://inseq.readthedocs.io/) for more details.

Answers:

**Task 1: Negation**
The results show that the way negation is expressed does affect the model's attributions. In the sentence with "do not," the negation is spread over several tokens with "not" carrying a moderate weight, which leads to a contrast probability difference of negative 0.054 and a higher overall probability. In the sentence using "false," the effect is much more concentrated, with "false" contributing a larger attribution of 0.167 and a much greater contrast probability difference of negative 0.399 along with a lower overall probability. This would suggest that using "false" produces a more pronounced shift in the model's predictions. However, when testing an example without negation ("Is it true that cows have six legs") the model seems similarly confident (0.453) as the example with negation (0.469). It could be that the model generally doesn't handle false information well.

**Task 2: Translating**
It is likely that the model relies on patterns learned from its training data, looking at cues in the source sentences and the translation prompt to decided which gender form is most likely. In the tests the heatmaps show that the nouns (cat, moon) are having the strongest effect.

**Task 3: Parameters**
It seems that the larger the models get, the more the models are confident about each word in the input. The small model places high emphasis on a few key tokens such as "equal" but remains less certain overall (0.264). The base model spreads attention more evenly across the input and has higher confidence (0.523). Lastly, the large model has even more uniform importance for each token, but has significantly lower confidence than the base model (0.238). This could suggest that model size correlates with increased token importance, but not overall confidence.

In [8]:
import inseq

def attribution(model, input, target, contrast):
  model = inseq.load_model(model, "input_x_gradient")
  out = model.attribute(
      input,
      target,
      attributed_fn="contrast_prob_diff",
      step_scores=["contrast_prob_diff", "probability"],
      contrast_targets=contrast,
  )
  out.show()

def main():
  print("Task 1: Negation")
  attribution(
      model="google/flan-t5-base",
      input="Is it true that cows do not have six legs?",
      target="yes",
      contrast="no"
  )
  print("Task 1: Negation")
  attribution(
      model="google/flan-t5-base",
      input="Is it false that cows have six legs?",
      target="yes",
      contrast="no"
  )
  print("Task 1: Negation")
  attribution(
      model="google/flan-t5-base",
      input="Is it true that cows have six legs?",
      target="yes",
      contrast="no"
  )


if __name__ == '__main__':
  main()

Task 1: Negation


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:02<00:00,  1.02s/it]


,▁no → ▁yes,</s>
▁I,0.052,0.0
s,0.04,0.0
▁it,0.057,0.0
▁true,0.107,0.0
▁that,0.088,0.0
▁cow,0.085,0.0
s,0.022,0.0
▁do,0.055,0.0
▁not,0.095,0.0
▁have,0.058,0.0


Task 1: Negation


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:01<00:00,  1.09it/s]


,▁no → ▁yes,</s>
▁I,0.072,0.0
s,0.072,0.0
▁it,0.059,0.0
▁false,0.167,0.0
▁that,0.057,0.0
▁cow,0.098,0.0
s,0.035,0.0
▁have,0.045,0.0
▁six,0.1,0.0
▁legs,0.129,0.0


Task 1: Negation


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:01<00:00,  1.06it/s]


,▁no → ▁yes,</s>
▁I,0.056,0.0
s,0.041,0.0
▁it,0.05,0.0
▁true,0.108,0.0
▁that,0.086,0.0
▁cow,0.115,0.0
s,0.034,0.0
▁have,0.055,0.0
▁six,0.147,0.0
▁legs,0.159,0.0


In [6]:
import inseq

def attribution(model, input, target, contrast):
  model = inseq.load_model(model, "input_x_gradient")
  out = model.attribute(
      input,
      target,
      attributed_fn="contrast_prob_diff",
      step_scores=["contrast_prob_diff", "probability"],
      contrast_targets=contrast,
  )
  out.show()

def main():
  print("Task 2: Translating")
  attribution(
      model="google/flan-t5-base",
      input="Translate into German: The cat went to the moon.",
      target="Die Katze ging zum Mond.",
      contrast="Eine Katze"
  )
  print("Task 2: Translating")
  attribution(
      model="google/flan-t5-base",
      input="Translate into French: The cat went to the moon.",
      target="Le chat est allé sur la lune.",
      contrast="Un chat"
  )
  print("Task 2: Translating")
  attribution(
      model="google/flan-t5-base",
      input="Translate into Italian: The cat went to the moon.",
      target="Il gatto è andato sulla luna.",
      contrast="Il gatto"
  )

if __name__ == '__main__':
  main()

Task 2: Translating


Attributing with input_x_gradient...: 100%|██████████| 10/10 [00:10<00:00,  1.18s/it]


,</s> → ▁Die,</s> → ▁Katz,</s> → e,</s> → ▁ging,</s> → ▁zum,▁Eine → ▁Mon,▁Katz → d,e → .,</s>
▁Translat,0.07,0.052,0.054,0.054,0.048,0.053,0.056,0.061,0.0
e,0.024,0.021,0.024,0.021,0.021,0.013,0.015,0.02,0.0
▁into,0.037,0.034,0.031,0.033,0.029,0.022,0.022,0.033,0.0
▁German,0.104,0.073,0.09,0.073,0.057,0.045,0.045,0.087,0.0
:,0.051,0.038,0.041,0.037,0.027,0.023,0.035,0.039,0.0
▁The,0.111,0.062,0.052,0.047,0.032,0.019,0.029,0.038,0.0
▁cat,0.242,0.388,0.334,0.095,0.067,0.043,0.053,0.083,0.0
▁went,0.055,0.055,0.11,0.19,0.13,0.05,0.067,0.091,0.0
▁to,0.024,0.025,0.033,0.083,0.129,0.058,0.068,0.071,0.0
▁the,0.015,0.014,0.016,0.032,0.042,0.048,0.041,0.044,0.0


Task 2: Translating


Attributing with input_x_gradient...: 100%|██████████| 13/13 [00:13<00:00,  1.10s/it]


,</s> → ▁Le,</s> → ▁chat,</s> → ▁est,</s> → ▁all,</s> → é,</s> → ▁sur,</s> → ▁la,</s> → ▁,</s> → l,▁Un → une,▁chat → .,</s>
▁Translat,0.082,0.035,0.046,0.056,0.065,0.042,0.046,0.039,0.052,0.041,0.071,0.0
e,0.03,0.017,0.017,0.024,0.022,0.017,0.021,0.014,0.014,0.018,0.027,0.0
▁into,0.04,0.02,0.029,0.037,0.04,0.022,0.029,0.022,0.022,0.028,0.033,0.0
▁French,0.133,0.05,0.058,0.077,0.091,0.05,0.067,0.047,0.048,0.079,0.097,0.0
:,0.056,0.031,0.035,0.039,0.052,0.023,0.031,0.022,0.023,0.025,0.037,0.0
▁The,0.113,0.057,0.043,0.037,0.049,0.027,0.029,0.028,0.02,0.025,0.032,0.0
▁cat,0.208,0.456,0.127,0.086,0.175,0.058,0.049,0.054,0.048,0.059,0.078,0.0
▁went,0.068,0.046,0.218,0.148,0.105,0.108,0.072,0.057,0.068,0.056,0.074,0.0
▁to,0.03,0.027,0.08,0.124,0.061,0.138,0.062,0.056,0.099,0.052,0.088,0.0
▁the,0.014,0.02,0.029,0.031,0.028,0.045,0.04,0.052,0.043,0.043,0.034,0.0


Task 2: Translating


Attributing with input_x_gradient...: 100%|██████████| 17/17 [00:18<00:00,  1.18s/it]


,</s> → ▁Il,</s> → ▁,</s> → gat,</s> → to,</s> → ▁,</s> → è,</s> → ▁and,</s> → a,</s> → to,</s> → ▁,</s> → s,▁Il → ul,▁ → la,gat → ▁luna,to → .,</s>
▁Translat,0.07,0.046,0.054,0.075,0.058,0.061,0.051,0.082,0.076,0.063,0.061,0.06,0.068,0.049,0.082,0.0
e,0.027,0.02,0.016,0.025,0.031,0.031,0.02,0.03,0.022,0.025,0.023,0.02,0.02,0.013,0.022,0.0
▁into,0.044,0.026,0.035,0.043,0.041,0.039,0.031,0.047,0.043,0.037,0.034,0.033,0.037,0.03,0.045,0.0
▁Italian,0.144,0.058,0.089,0.088,0.084,0.098,0.072,0.145,0.121,0.077,0.07,0.08,0.12,0.086,0.108,0.0
:,0.045,0.031,0.033,0.035,0.033,0.037,0.033,0.037,0.041,0.022,0.029,0.034,0.028,0.022,0.05,0.0
▁The,0.104,0.069,0.052,0.051,0.042,0.052,0.042,0.045,0.043,0.028,0.031,0.031,0.028,0.022,0.035,0.0
▁cat,0.246,0.458,0.462,0.349,0.135,0.086,0.102,0.109,0.126,0.073,0.055,0.071,0.061,0.042,0.064,0.0
▁went,0.059,0.048,0.045,0.075,0.2,0.223,0.224,0.142,0.12,0.117,0.138,0.094,0.067,0.054,0.066,0.0
▁to,0.032,0.018,0.02,0.029,0.091,0.084,0.12,0.055,0.077,0.113,0.153,0.076,0.059,0.043,0.057,0.0
▁the,0.015,0.012,0.014,0.025,0.031,0.024,0.027,0.032,0.026,0.037,0.048,0.036,0.033,0.048,0.038,0.0


In [7]:
import inseq

def attribution(model, input, target, contrast):
  model = inseq.load_model(model, "input_x_gradient")
  out = model.attribute(
      input,
      target,
      attributed_fn="contrast_prob_diff",
      step_scores=["contrast_prob_diff", "probability"],
      contrast_targets=contrast,
  )
  out.show()

def main():
  print("Task 3: Parameters-Small")
  attribution(
      model="google/flan-t5-small",
      input="Does 3 + 3 equal 6?",
      target="yes",
      contrast="no"
  )
  print("Task 3: Parameters-Base")
  attribution(
      model="google/flan-t5-base",
      input="Does 3 + 3 equal 6?",
      target="yes",
      contrast="no"
  )
  print("Task 3: Parameters-Large")
  attribution(
      model="google/flan-t5-large",
      input="Does 3 + 3 equal 6?",
      target="yes",
      contrast="no"
  )

if __name__ == '__main__':
  main()

Task 3: Parameters-Small


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:00<00:00,  2.54it/s]


,▁no → ▁yes,</s>
▁Does,0.16,0.0
▁3,0.096,0.0
▁+,0.116,0.0
▁3,0.072,0.0
▁equal,0.251,0.0
▁6,0.093,0.0
?,0.114,0.0
</s>,0.097,0.0
contrast_prob_diff,-0.354,-0.0
probability,0.264,1.0


Task 3: Parameters-Base


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:01<00:00,  1.20it/s]


,▁no → ▁yes,</s>
▁Does,0.149,0.0
▁3,0.09,0.0
▁+,0.089,0.0
▁3,0.097,0.0
▁equal,0.191,0.0
▁6,0.154,0.0
?,0.11,0.0
</s>,0.12,0.0
contrast_prob_diff,0.05,-0.0
probability,0.523,1.0


Task 3: Parameters-Large


Attributing with input_x_gradient...: 100%|██████████| 3/3 [00:09<00:00,  4.59s/it]


,▁no → ▁yes,</s>
▁Does,0.132,0.0
▁3,0.107,0.0
▁+,0.134,0.0
▁3,0.109,0.0
▁equal,0.155,0.0
▁6,0.091,0.0
?,0.094,0.0
</s>,0.178,0.0
contrast_prob_diff,-0.237,-0.0
probability,0.238,1.0
